In [1]:
import spacy
import random
import json
from spacy.tokens import DocBin
from spacy.tokenizer import Tokenizer
import wandb
wandb.login()

### Ouverture des fichiers annotés et séparation en deux sous-corpus

In [55]:
with open('corpus_annote/doccano_serie0.jsonl', encoding='utf-8') as f:
    result = [json.loads(jline) for jline in f.read().splitlines()]

random.shuffle(result)
training_corpus = result[:220]
test_corpus = result[220:]

In [7]:
nlp = spacy.load("fr_core_news_lg")

In [57]:
def create_training(TRAIN_DATA):
    db = DocBin()
    for training_example in TRAIN_DATA:
        text = training_example['data']
        labels = training_example['label']
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in labels:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents 
        db.add(doc)
    print(len(db))
    return(db)

In [ ]:
train = create_training(training_corpus)
train.to_disk("train_serie1.spacy")

valid = create_training(test_corpus)
valid.to_disk("valid_serie1.spacy")

In [ ]:
!python -m spacy init fill-config configs/base_config.cfg configs/config.cfg

In [61]:
!python -m spacy train configs/config.cfg --output ./output --paths.train ./train_serie1.spacy --paths.dev ./valid_serie1.spacy
#43 minutes to run (16go RAM et quadcore mais pas de multiprocessing)

[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    154.50    0.31    0.18    1.13    0.00
  0     200       8953.36  13278.15    5.84   11.67    3.89    0.06
  1     400        835.93   5396.31   32.78   45.21   25.72    0.33
  2     600        501.31   4796.94   36.30   33.64   39.42    0.36
  3     800        159.49   4445.30   38.51   68.76   26.75    0.39
  4    1000        117.74   3228.30   36.17   68.01   24.64    0.36
  5    1200        217.04   4170.59   39.56   46.88   34.21    0.40
  6    1400        209.38   3653.36   41.74   57.24   32.85    0.42
  7    1600        628.28   3299.07   42.54   40.07   45.33 

2022-03-03 16:43:09.469678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-03 16:43:09.469710: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Set up nlp object from config
Pipeline: ['tok2vec', 'ner']
Created vocabulary
Added vectors: fr_core_news_lg
Finished initializing nlp object
Initialized pipeline components: ['tok2vec', 'ner']


In [66]:
nlp_ner = spacy.load("output/model-best")

In [68]:
structures_adm = ["commune", "Commune", "district", "Département", "département", "départemens", "Départemens", "municipalité", "Municipalité", "ville", "village", "canton"
         ]
structures_adm2 = ["commune", "Commune", "district", "Département", "département", "départemens", "Départemens", "municipalité", "Municipalité", "ville", "village", "canton"
         ]
lieux = ['Basse-Alpes', 'Maine-&-Loire', 'Seine', 'Alpes-maritimes', 'Arriége', 'Charente-inférieure', 'Côte-d\'or', 'Côtes-du-Nord', 'Dyle', 'Escaut', 'Finistère', 'Forêts', 'Haute-Garonne', 'Ille-et-vilaine', 'Indre-et-Loire', 'Jemmapes', 'Loire-inférieure', 'Lys', 'Meurthe', 'Meuse-inférieure', 'Deux-Nèthes', 'Ourthe', 'Puy-de-Dôme', 'Basses-Pyrénées', 'Pyrénées-Orientales', 'Sambre-et-Meuse', 'Haute-Saone', 'Saone-et-Loire', 'Seine-inférieure', 'Deux-Sèvres', 'La Souterraine', 'Alès', 'Moulins-Engilbert', 'Segré', 'La Charité', 'Nérac', 'Bitche', 'La Ferté-Bernard', 'Toulon', 'Pont-de-Vaux', 'Châtellerault', 'Soissons', 'Noyon', 'Chartres', 'Bourbonne-les-Bains', 'Crépy', 'Autun', 'Mauléon', 'Mirande', 'Tartas', 'Douai', 'Poitiers', 'Clamecy', 'Mirecourt', 'Vihiers', 'Nogent-Le-Rotrou', 'Janville', 'Céret', 'Nevers', 'Digne', 'Mer', 'Saint-Claude', 'Brest', "L'Isle-Jourdain", 'Saint-Gaudens', 'Pontoise', 'Montlieu', 'Limoux', 'Strasbourg', 'Embrun', 'Fougères', 'Monflanquin', 'Calais', 'Provins', 'Montdidier', 'Saint-Marcellin', 'Bourges', 'Falaise', 'Vico', 'Caen', 'Barcelonnette', 'Fresnay-sur-Sarthes', 'Mur-de-Barrez', 'Saint-Rambert', 'Tulle', 'Arcis-sur-Aube', 'Chauny', 'La Rochefoucauld', 'Evreux', 'Vesoul', "Pont-l'Evêque", 'Crest', 'Vendôme', 'Loches', 'La Guerche', 'Mont-de-Marsan', 'Montmorillon', 'Corbeil', 'Avesnes', 'Villefort', 'Neuville-aux-Bois', 'Grand-Andelys', 'Cany', 'Guérêt', "Saint-Jean-d'Angély", 'Bazas', 'Nogaro', 'Breteuil', 'Amiens', 'Altkirch', 'Bellême', 'Melun', 'Commercy', 'Cervione', 'Lamarche', 'Oloron', 'Rodez', 'Saint-Céré AND Figeac', 'Brignoles', 'Argelès', 'Lisieux', 'Barbezieux', 'Amboise', 'Château-Thierry', 'Trévoux', 'Lesparre', 'Apt', 'Saint-Florentin', 'Meyrueis', 'Broons', 'Châteaudun', 'Châtillon-sur-Seine', 'Charolles', 'Thouars', 'Bergerac', 'Lure', 'Monistrol', 'Tonneins', 'Le Blanc', 'Mamers', 'Château-du-Loir', 'Blamont', 'Cognac', 'Billom', 'Gex', 'Laon', 'Etampes', 'Tournon Le Mézenc', 'Le Faouët', 'Dinan', 'Loudéac', 'Bruyères', 'Saint-Flour', 'Sens', 'Wissembourg', 'Challans', 'Saint-Chély', 'Bellac', 'Jussey', 'Paris', 'Ajaccio', 'Pithiviers', 'Bar-le-Duc', 'Cadillac', 'Joyeuse La Tanargue', 'Verdun', 'Carentan', 'Lagrasse', 'Loudun', 'Sancoins', 'Redon', 'Bourg', 'Ploërmel', 'Cosne', 'Valenciennes', 'Saint-Pol', 'Serres', 'Riom', 'Montmarault', 'Bellay', 'Avallon', 'Vire', 'Montivilliers', 'La Porta', 'Romorantin', 'Grandpré', 'Saint-Leonard', 'Cérilly', 'Bain', 'Sancerre', 'Ruffec', 'Alençon', 'Montaigu', 'La Roche-Bernard', 'Montbrisson', 'Roanne', 'Bar-sur-Aube', 'Tallano', 'Bernay', 'Remiremont', 'Saint-Denis', 'Hazebrouck', 'Gap', 'Confolens', 'Châlons', 'Tours', 'Langogne', 'Evaux', 'Perpignan', 'Ernée', 'Benfeld', 'Montreuil', 'Sedan', 'Langeais', 'Aubusson', 'La-Barthe-de-Neste', 'Béthune', 'Gaillac', 'Saint-Paul', 'Sarlat', 'Figeac', 'Florac', 'Ribérac', 'Dieuze', 'Bar-sur-Seine', 'Aurillac', 'Mirepoix', 'Mondoubleau', 'Josselin', 'Saint-Aignan', 'Dieppe', 'Ussel', 'Aubigny', 'Chateau-Renault', 'Machecoul', 'Gray', 'Versailles', 'Grasse', 'Libourne', 'Châteauroux', 'Castres', 'Marcigny', 'Arles', 'Marennes', 'Lectoure', 'Saint-Maximin', 'Narbonne', 'Saint-Affrique', 'Caudebec', 'Excideuil', 'Cholet', 'Lauzun', 'Belvès', 'Pont-Audemer', 'Saint-Yrieix', 'Rethel', 'Romans', 'Besse', 'Montluel', 'Coutances', 'Darney', 'Epinal', 'Dreux', 'Chateaubriant', 'Privas', 'Guérande', 'Orleans', 'Château-Salins', 'Moulins', 'La Rochelle', 'Saint-Dizier', 'Angers', 'Argenton', 'Saint-Germain', 'Carhaix', 'Mende', 'Ile-Rousse', 'Corte', 'Orthez', 'Vaucouleurs', 'Lacaune', 'Clermont-en-Argonne', 'Saint-Palais', "Saint-Geniez-de-Rive-d'Olt", 'Chateauneuf', 'Baugé', 'Bourganeuf', 'Château-Chinon', 'Vierzon', 'Muret', 'Luxeuil', 'Sillé-le-Guillaume', 'Bourg-en-Bresse', 'Bourmont', 'Rozay', 'Lavaur', 'Charleville', 'Beaucaire', 'Sisteron', 'Pont-à-Mousson', 'Dax', 'Fréjus', 'Is-sur-Tille', 'Bordeaux', 'Belfort', 'Saint-Amand', 'Savenay', 'Boulogne', 'Lusignan', 'Châteaumeillant', 'Pons', 'Nemours', 'Saint-Calais', 'Colmar', 'Montauban', 'La-Tour-du-Pin', 'Châtillon-sur-Indre', 'Marseille', 'Grenade', 'Marmande', 'Saint-Florent-le-Vieil', 'Sauveterre', 'Nogent-sur-Seine', 'Joigny', 'Montargis', 'Nyons', 'Arnay-le-Duc', 'Vitry-le-François', 'Pont Saint-Esprit', 'Saint-Junien', 'Draguignan', 'Casteljaloux', 'Montignac', 'Stenay', 'Lunéville', 'Montluçon', 'Melle', 'Clisson', 'Sablé', 'Lauzerte', 'Saint-Maixent', 'Brive-la-Gaillarde', 'Joinville', 'Châtillon-sur-Saône', 'Tarbes', 'Neufchâteau', 'Domfront', 'Briey', 'Castellane', 'Gournay', 'Castelsarrasin', 'Le Puy', 'Mauriac', 'Vannes', 'Paimbeuf', 'Montélimar', 'Ustarritz', 'Mâcon', 'Nimes', 'Agen', 'Saint-Quentin', 'Murat', 'Quingey', 'Dijon', 'Louviers', 'Vienne', 'Chaumont', 'Etain', 'Saint-Omer', 'Cherbourg', 'Sainte-Sever', 'Auxerre', 'Le Donjon', 'Mussidan', 'Dol', 'La Flèche', 'Bourbon-Lancy', 'Barjols', 'Epernay', "Villeneuve-d'Agen", 'Aubin', 'Vitré', 'Prades', 'Pau', 'Châteauneuf', 'Tonnerre', 'Poligny', 'Doullens', 'Vervins', 'Foix', 'Rennes', 'Château-Gontier', 'Blain', 'Châteaulin', 'Le Quesnoy', 'Revel', 'Thionville', 'Orgelet', 'Nantes', 'Millau', 'Haguenau', 'Parthenay', 'Saint-Brieuc', 'Issoudun', 'Blois', 'Toulouse', 'Le Dorat', 'Langres', 'Metz', "L'Aigle", 'Sarrelouis', 'Saint-Jean-de-Losne', 'Bourg-la-Reine', 'Vouziers', 'Sommières', 'Saint-Céré', 'La Châtaigneraie', 'Pontrieux', 'Lamballe', 'Rouen', 'Saint-Hippolyte', 'Mortain', 'Uzès', 'Evron', 'Die', 'Bastia', 'Issoire', 'Uzerche', 'Sainte-Menehould', 'Boussac', 'Nontron', 'Brioude', 'Le Mans', 'Bapaume', 'Saumur', 'Besançon', 'Saint-Mihiel', 'Chinon', 'Bayeux', 'Saintes', 'Decize', 'Fontenay-le-Comte', 'Péronne', 'Valognes', 'Saint-Girons', 'Arbois', 'Grandvilliers', 'Rostrenen', 'Verneuil', 'Lesneven', 'Cahors', 'Condom', 'Saint-Lô', 'Cusset', 'Limoges', 'Toul', 'Lyon', 'Avranches', 'Thiers', 'Corbigny', 'Felletin', 'Pontecroix', 'Beaugency', 'Lille', 'Auray', 'Longwy', 'Vic-Bigorre', 'Ornans', 'Nancy', 'Saint-Fargeau', 'Béziers', 'Montaigut', 'Morlaix', 'Clermont', 'Reims', 'Neufchâtel', 'Hennebont', 'Salon-de-Provence', 'Saint-Malo', 'Ancenis', 'Saint-Pons-de-Thomières', 'Semur-en-Auxois', 'Abbeville', 'Pontivy', 'Gien', 'Argentan', 'Rocroi', 'Briançon', 'Castelnaudary', 'Auch', 'Beauvais', 'Tarascon', 'Saint-Pierre-le-Moûtier', 'Vézelise', 'Mantes', 'Gomesse', 'Châlon-sur-Saône', 'Meaux', 'Angoulême', "Les Sables-d'Olonne", 'Lons-le-Saunier', 'Arras', 'Mortagne', 'Aix-en-Provence', 'Séverac-le-Château', 'Lannion', 'Montpellier', 'Quillan', 'Boulay', 'Clermont-Ferrand', 'Le vigan', 'Dourdan', 'Carcassonne', 'Sézanne', 'Quimper', 'Laval', 'La Roche-sur-Yon', 'Rieux', 'Compiègne', 'Louhans', 'Preuilly', 'Civray', 'Landerneau', 'Rambervilliers', 'Villaines', 'Sarreguemines', 'Baume-les-Dames', 'Champlitte', 'Gourdon', 'Boiscommun', 'Saint-Etienne', 'Nantua', 'Ambert', 'La Châtre', 'Bergues', 'La Réole', 'Beaune', 'Niort', 'Gannat', 'Orange', 'Oletta', 'Hyères', 'Pontarlier', 'Guimgamp', 'Quimperlé', 'Cambrai', 'Albi', 'Mayenne', 'Bagnères-de-Bigorre', 'Saint-Giirons', 'Dôle', 'Troyes', 'Châtillon-les-Dombes', 'Craon', 'Marvejols', 'Périgueux', 'Forcalquier', 'Ervy', 'Saint-Dié', 'Grenoble', 'Morhange', 'Sarrebourg', 'Senlis', 'Lodève', 'Montfort', 'Rochefort', 'Villefranche', 'Valence', 'Montbrison', 'Montélimard', 'Menton', 'Delémont', 'Senones', 'Lodèye', 'Saint-Jean-de-Maurienne', 'rocroi', 'Montbéliard', 'Chambéry', 'Carouge', 'Mézières', 'Moutiers', 'Thonon', 'Avignon', 'Carpentras', 'Neu-Saarwerden', 'Neu-Sarrewerden', 'Puget-Théniers', 'Porrentruy', 'Cluses', 'Nice', 'Annecy', 'Creuse', 'Gard', 'Nièvre', 'Maine-et-Loire', 'Lot-et-Garonne', 'Moselle', 'Sarthe', 'Var', 'Ain', 'Aisne', 'Oise', 'Eure-et-Loir', 'Haute-Marne', 'Saône-et-Loire', 'Pyrénées-Atlantiques', 'Gers', 'Landes', 'Nord', 'Vosges', 'Pyrénées-Orientales', 'Alpes-de-Haute-Provence', 'Loir-et-Cher', 'Jura', 'Finistère', 'Haute-Garonne', 'Seine-et-Oise', 'Charente-Maritime', 'Aude', 'Bas-Rhin', 'Hautes-Alpes', 'Ille-et-Vilaine', 'Pas-de-Calais', 'Seine-et-Marne', 'Somme', 'Isère', 'Cher', 'Calvados', 'Corse', 'Aveyron', 'Corrèze', 'Aube', 'Charente', 'Eure', 'Haute-Saône', 'Drôme', 'Indre-et-Loire', 'Lozère', 'Loiret', 'Seine-Maritime', 'Gironde', 'Haut-Rhin', 'Orne', 'Meuse', 'Lot', 'Hautes-Pyrénées', 'Bouches-du-Rhône', 'Yonne', "Côtes-d'Armor", "Côte-d'Or", 'Deux-Sèvres', 'Dordogne', 'Haute-Loire', 'Indre', 'Meurthe-et-Moselle', 'Puy-De-Dôme', 'Ardèche', 'Morbihan', 'Cantal', 'Vendée', 'Haute-Vienne', 'Manche', 'Allier', 'Ardennes', 'Rhône-et-Loire', 'Marne', 'Tarn', 'Alpes-Maritimes','Alpes Maritimes', 'Ariège', 'Loire-Atlantique', 'Doubs', 'Hérault', 'Drome', 'Loire', 'Golo', 'Vaucluse', 'Mont-Terrible', 'Liamone', 'Mont-Blanc', 'Rhône', 'Seine & Oise', 'Seine & Marne', 'Seine-Inférieure', 'Seine - Inférieure','Seine- Inférieure', 'Léman', 'Côtes-du-Nord'
         ]
structures_locales = ["canton", "administration municipale", "assemblée communale"]
patt = []
#lemme-ADP-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "ADP"}, 
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_adp_lemme)

#lemme-DET-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"}, 
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_lemme)
        
#lemme-ADP-DET-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_adp_det_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "ADP"},  
                 {"POS": "DET"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_adp_det_lemme)
        
#lemme-DET-ADP-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"},  
                 {"POS": "ADP"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_adp_lemme)

#lemme-DET-DET-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_det_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"},  
                 {"POS": "DET"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_det_lemme)

#Lemme-ADP-ADP-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_adp_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "ADP"},  
                 {"POS": "ADP"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_adp_adp_lemme)

#structures_locales-ADP-nom propre
for adm in structures_locales:
    patt_structures_locales_adp = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "ADP"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_adp)

#structures_locales_DET-nom propre
for adm in structures_locales:
    patt_structures_locales_det = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "DET"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_det)

#structures_locales_ADP_DET_ nom propre
for adm in structures_locales:
    patt_structures_locales_adp_det = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "ADP"},
            {"POS": "DET"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_adp_det)

#structures_locales_ADP_ADP_nom_propre
for adm in structures_locales:
    patt_structures_locales_adp_adp = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "ADP"},
            {"POS": "ADP"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_adp_adp)

#structures_locales_DET_ADP_nom_propre
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"},  
                 {"POS": "ADP"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_adp_lemme)

#structures_locales_DET_DET_nom_propre
for adm in structures_locales:
    patt_structures_locales_det_det = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "DET"},
            {"POS": "DET"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_det_det)

#inversion_du_sujet_DET
for adm in structures_adm:
    for adm2 in structures_adm2:
        patt_inversion_du_sujet1 = {
            "label": "GPE",
            "pattern": [
            {"POS": "PROPN"},
            {"TEXT": adm},
            {"POS": "DET"},
            {"TEXT": adm2},
            ],
        }
        patt.append(patt_inversion_du_sujet1)
        
#inversion_du_sujet_ADP
for adm in structures_adm:
    for adm2 in structures_adm2:
        patt_inversion_du_sujet2 = {
            "label": "GPE",
            "pattern": [
            {"POS": "PROPN"},
            {"TEXT": adm},
            {"POS": "ADP"},
            {"TEXT": adm2},
            ],
        }
        patt.append(patt_inversion_du_sujet2)
        
nlp_ner = spacy.load("output/model-best")

#Ajout d'une règle spéciale pour les traits d'union (Hyphens)
infixes = nlp_ner.Defaults.prefixes + ([r"[./]", r"[-]~", r"[&]~", r"(.'.)"])
infix_re = spacy.util.compile_infix_regex(infixes)

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)

nlp_ner.tokenizer = custom_tokenizer(nlp_ner)

#Déplacement de l'entity ruler avant la NER (priorisation de l'entity ruler)
ruler = nlp_ner.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(patt)

In [69]:
def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
            print("\n")   

In [70]:
#Pour comparer notre premier modèle avec le modèle de NLP original
nlp = spacy.load("fr_core_news_md")

In [76]:
nlp_ner.add_pipe("morphologizer", source=spacy.load("fr_core_news_md"), before="entity_ruler")
nlp_ner.add_pipe("parser", source=spacy.load("fr_core_news_md"), before="entity_ruler")
nlp_ner.add_pipe("attribute_ruler", source=spacy.load("fr_core_news_md"), before="entity_ruler")
nlp_ner.add_pipe("lemmatizer", source=spacy.load("fr_core_news_md"), before="entity_ruler")

### Comparaison entre le NLP_NER que l'on a entraîné et le NLP de base

In [ ]:
with open('test_nouveau_corpus/urgence2.jsonl', encoding='utf-8') as f:
    test_dataset = [json.loads(jline) for jline in f.read().splitlines()]
for test in test_dataset[1:20]:
    text = test['text']
    doc = nlp_ner(text)
    show_ents(doc)

In [ ]:
with open('test_nouveau_corpus/urgence2.jsonl', encoding='utf-8') as f:
    test_dataset = [json.loads(jline) for jline in f.read().splitlines()]
for test in test_dataset[1:20]:
    text = test['text']
    doc = nlp(text)
    show_ents(doc)